## Text Splitters 文本分割器

加载文档后，您通常会想要对其进行转换以更好地适合您的应用程序。最简单的例子是，您可能希望将长文档分割成更小的块，以适合模型的上下文窗口。 LangChain 有许多内置的文档转换器，可以轻松地拆分、组合、过滤和以其他方式操作文档。

当您想要处理长文本时，有必要将该文本分割成块。这听起来很简单，但这里存在很多潜在的复杂性。理想情况下，您希望将语义相关的文本片段保留在一起。 “语义相关”的含义可能取决于文本的类型。本笔记本展示了实现此目的的几种方法。

在较高层面上，文本分割器的工作原理如下：
- 将文本分成小的、具有语义意义的块（通常是句子）。
- 开始将这些小块组合成一个更大的块，直到达到一定的大小（通过某些函数测量）。
- 一旦达到该大小，请将该块设为自己的文本片段，然后开始创建具有一些重叠的新文本块（以保持块之间的上下文）。

### HTMLHeaderTextSplitter

“MarkdownHeaderTextSplitter”、“HTMLHeaderTextSplitter”是一个“结构感知”分块器，它在元素级别拆分文本，并为每个与任何给定块“相关”的标题添加元数据。它可以逐个元素返回块，或者将元素与相同的元数据组合起来，目的是 (a) 保持相关文本在语义上（或多或少）分组，以及 (b) 保留文档结构中编码的上下文丰富的信息。它可以与其他文本分割器一起使用，作为分块管道的一部分。

In [1]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./html/Animation-system.html",encoding="utf8")
doc = loader.load()
doc

[Document(page_content='<!DOCTYPE html>\n<html lang="zh">\n\t<head>\n\t\t<meta charset="utf-8" />\n\t\t<base href="../../../" />\n\t\t<script src="page.js"></script>\n\t\t<link type="text/css" rel="stylesheet" href="page.css" />\n\t</head>\n\t<body>\n\t\t<h1>动画系统（[name]）</h1>\n\n\t\t<h2>概述</h2>\n\n\t\t<p class="desc">\n\t\t\t在three.js动画系统中，您可以为模型的各种属性设置动画：\n\t\t\t[page:SkinnedMesh]（蒙皮和装配模型）的骨骼，morph targets（变形目标），\n\t\t\t不同的材料属性（颜色，不透明度，布尔运算），可见性和变换。动画属性可以淡入、淡出、交叉淡化和扭曲。\n\t\t\t在相同或不同物体上同时发生的动画的权重和时间比例的变化可以独立地进行。\n\t\t\t相同或不同物体的动画也可以同步发生。\n\t\t\t<br /><br />\n\t\t\t为了在一个同构系统中实现所有这一切，\n\t\t\tthree.js的动画系统[link:https://github.com/mrdoob/three.js/issues/6881 在2015年彻底改变]（注意过时的信息！），\n\t\t\t它现在有一个与Unity/虚幻4引擎类似的架构。此页面简要阐述了这个系统中的主要组件以及它们如何协同工作。\n\t\t</p>\n\n\t\t<h3>动画片段（Animation Clips）</h3>\n\n\t\t<p class="desc">\n\t\t\t如果您已成功导入3D动画对象（无论它是否有骨骼或变形目标或两者皆有都不要紧）——\n\t\t\t例如使用[link:https://github.com/KhronosGroup/glTF-Blender-IO glTF Blender exporter]（glTF Blender导出器）</a>\n\t\t\t从Blender导出它并使用[pa

In [8]:
headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

In [9]:
# pip install -qU langchain-text-splitters

In [10]:
from langchain_text_splitters import HTMLHeaderTextSplitter
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
html_header_splits = html_splitter.split_text(doc[0].page_content)
html_header_splits

[Document(page_content='在three.js动画系统中，您可以为模型的各种属性设置动画： [page:SkinnedMesh]（蒙皮和装配模型）的骨骼，morph targets（变形目标）， 不同的材料属性（颜色，不透明度，布尔运算），可见性和变换。动画属性可以淡入、淡出、交叉淡化和扭曲。 在相同或不同物体上同时发生的动画的权重和时间比例的变化可以独立地进行。 相同或不同物体的动画也可以同步发生。 为了在一个同构系统中实现所有这一切， three.js的动画系统[link:https://github.com/mrdoob/three.js/issues/6881 在2015年彻底改变]（注意过时的信息！）， 它现在有一个与Unity/虚幻4引擎类似的架构。此页面简要阐述了这个系统中的主要组件以及它们如何协同工作。', metadata={'Header 1': '动画系统（[name]）', 'Header 2': '概述'}),
 Document(page_content='如果您已成功导入3D动画对象（无论它是否有骨骼或变形目标或两者皆有都不要紧）—— 例如使用[link:https://github.com/KhronosGroup/glTF-Blender-IO glTF Blender exporter]（glTF Blender导出器） 从Blender导出它并使用[page:GLTFLoader]将其加载到three.js场景中 —— 其中一个响应字段应该是一个名为“animations”的数组， 其中包含此模型的[page:AnimationClip AnimationClips]（请参阅下面可用的加载器列表）。 每个*AnimationClip*通常保存对象某个活动的数据。 举个例子，假如mesh是一个角色，可能有一个AnimationClip实现步行循环， 第二个AnimationClip实现跳跃，第三个AnimationClip实现闪避等等。', metadata={'Header 1': '动画系统（[name]）', 'Header 2': '概述', 'Header 3': '动画片段（Animation Clips）'}),
 Document(page_content='在这样的*AnimationClip

In [11]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
embeddings_path = "D:\\ai\\download\\bge-large-zh-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=embeddings_path)

C:\Users\a4154\AppData\Roaming\Python\Python310\site-packages\transformers\utils\hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
d:\ProgramData\anaconda3\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [12]:
vectorStoreDB = FAISS.from_documents(html_header_splits,embedding=embeddings)
vectorStoreDB

In [13]:
from langchain_core.prompts import ChatPromptTemplate

template ="""
只根据以下文档回答问题：
{context}

问题：{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [14]:
retriever = vectorStoreDB.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 2}
)

In [15]:
docs = retriever.get_relevant_documents("请说说如何控制动画的播放和暂停")
docs

[Document(page_content='*AnimationMixer*本身只有很少的（大体上）属性和方法， 因为它可以通过[page:AnimationAction AnimationActions]来控制。 通过配置*AnimationAction*，您可以决定何时播放、暂停或停止其中一个混合器中的某个*AnimationClip*， 这个*AnimationClip*是否需要重复播放以及重复的频率， 是否需要使用淡入淡出或时间缩放，以及一些其他内容（例如交叉渐变和同步）。', metadata={'Header 1': '动画系统（[name]）', 'Header 2': '概述', 'Header 3': '动画行为（Animation Actions）'}),
 Document(page_content='如果您希望一组对象接收共享的动画状态，则可以使用[page:AnimationObjectGroup]。', metadata={'Header 1': '动画系统（[name]）', 'Header 2': '概述', 'Header 3': '动画对象组（Animation Object Groups）'})]

In [16]:
from langchain_core.runnables import RunnableParallel,RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

outputParser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {
        "context":retriever,
        "question":RunnablePassthrough()
    }
)

result = setup_and_retrieval.invoke("请说说如何控制动画的播放和暂停？")
result

{'context': [Document(page_content='*AnimationMixer*本身只有很少的（大体上）属性和方法， 因为它可以通过[page:AnimationAction AnimationActions]来控制。 通过配置*AnimationAction*，您可以决定何时播放、暂停或停止其中一个混合器中的某个*AnimationClip*， 这个*AnimationClip*是否需要重复播放以及重复的频率， 是否需要使用淡入淡出或时间缩放，以及一些其他内容（例如交叉渐变和同步）。', metadata={'Header 1': '动画系统（[name]）', 'Header 2': '概述', 'Header 3': '动画行为（Animation Actions）'}),
  Document(page_content='如果您希望一组对象接收共享的动画状态，则可以使用[page:AnimationObjectGroup]。', metadata={'Header 1': '动画系统（[name]）', 'Header 2': '概述', 'Header 3': '动画对象组（Animation Object Groups）'})],
 'question': '请说说如何控制动画的播放和暂停？'}

In [17]:
from langchain_openai import ChatOpenAI, OpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://127.0.0.1:1234/v1"
model = ChatOpenAI(
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    temperature=0.3,
)


In [19]:
chain = setup_and_retrieval | prompt | model | outputParser
chain.invoke("请说说如何控制动画的播放和暂停？")

'要控制动画的播放、暂停或停止，您需要使用[page:AnimationAction]。通过配置*AnimationAction*，您可以决定何时播放、暂停或停止其中一个混合器中的某个*AnimationClip*。'